In [1]:
from itertools import product
from copy import deepcopy

In [2]:
def isLinearlyIndependent(S):
    M = matrix([list(t) for t in S])
    if len(S) < n+1:
        if M.rank() == len(S):
            return True
        return False
    else:
        if any(l == 0 for l in M.minors(n+1)):
            return False
        return True

In [3]:
def calculate_periodic_points(f, g):
    return Set(f.periodic_points(1)), Set(g.periodic_points(1))

In [4]:
def initialize_dictionaries(f, g, L, K, mult_to_point_L, mult_to_point_K, point_to_mult_L, point_to_mult_K, repeated_mult_L):
    for i in range(len(L)):
        mult_L = f.multiplier(L[i], 1).charpoly()
        mult_K = g.multiplier(K[i], 1).charpoly()
        tup_L = (mult_L, level)
        tup_K = (mult_K, level)
        if tup_L not in mult_to_point_L:
            mult_to_point_L[tup_L] = [L[i]]
        else:
            mult_to_point_L[tup_L] += [L[i]]
        if tup_K not in mult_to_point_K:
            mult_to_point_K[tup_K] = [K[i]]
        else:
            mult_to_point_K[tup_K] += [K[i]]
        point_to_mult_L[L[i]] = (mult_L, 0)
        point_to_mult_K[K[i]] = (mult_K, 0)
    for mult_L in mult_to_point_L:
        repeated = len(mult_to_point_L[mult_L])
        if mult_L not in mult_to_point_K:
            pass
        elif len(mult_to_point_K[mult_L]) != repeated:
            pass
        if repeated not in repeated_mult_L:
            repeated_mult_L[repeated] = [mult_to_point_L[mult_L]]
        else:
            repeated_mult_L[repeated] += [mult_to_point_L[mult_L]]

In [5]:
def find_independent_set(repeated_mult_L, source, corresponding):
    more = True
    for r in repeated_mult_L:
        for point_lst in repeated_mult_L[r]:
            for point in point_lst:
                if isLinearlyIndependent(source + [point]):
                    source.append(point)
                    mult = point_to_mult_L[point]
                    # if another point with this multiplier and level pair is in S
                    # then the multiplier level pair will be the last element of corresponding
                    if len(corresponding) != 0:
                        if corresponding[-1][0] == mult:
                            corresponding[-1][1] += 1
                        else:
                            corresponding.append([mult, 1])
                    else:
                        corresponding.append([mult, 1])
                if len(source) == n+2:
                    more = False
                    break
            if len(source) == n+2:
                break
        if len(source) == n+2:
            break
    return more

In [6]:
def find_preimages(i_repeated_mult, point_to_mult_L, mult_to_point_K, f, g, mult_to_point_L, point_to_mult_K, repeated_mult_L, a_repeated_mult):
    for r in i_repeated_mult:
            for point_lst_L in i_repeated_mult[r]:
                old_tup_L = point_to_mult_L[point_lst_L[0]]
                #print('current tup:', old_tup_L)
                point_lst_K = mult_to_point_K[old_tup_L]
                mult_L = old_tup_L[0]
                Tl = []
                Tk = []
                for pnt in point_lst_L:
                    for preimage in f.rational_preimages(pnt):
                        if preimage != pnt:
                            Tl.append(preimage)
                for pnt in point_lst_K:
                    for preimage in g.rational_preimages(pnt):
                        if preimage != pnt:
                            Tk.append(preimage)
                #print('Tl:', Tl)
                #print('Tk:', Tk)
                if len(Tl) != len(Tk):
                    return []
                if len(Tl) != 0:
                    found_no_more = False
                    new_tup_L = (mult_L, level)
                    new_tup_K = (mult_L, level)
                    #print('new_tup_L:', new_tup_L)
                    mult_to_point_L[new_tup_L] = Tl
                    mult_to_point_K[new_tup_K] = Tk
                    for i in range(len(Tl)):
                        point_to_mult_L[Tl[i]] = new_tup_L
                        point_to_mult_K[Tk[i]] = new_tup_K
                    repeated = len(Tl)
                    if repeated not in repeated_mult_L:
                        repeated_mult_L[repeated] = [Tl]
                    else:
                        repeated_mult_L[repeated] += [Tl]
                    if repeated not in a_repeated_mult:
                        a_repeated_mult[repeated] = [Tl]
                    else:
                        a_repeated_mult[repeated] += [Tl]
                    # right now rechecking independence
                    source = []
                    corresponding = []
                    more = find_independent_set(repeated_mult_L, source, corresponding)
                if not more:
                    break
            if not more:
                break
    return more, found_no_more, source, corresponding

In [7]:
def initialize_interator(corresponding, possible_targets, mult_to_point_K, arrangements_iterators):
    for tup in corresponding:
        possible_targets.append([mult_to_point_K[tup[0]], tup[1]])
    #print('possible_targets:', possible_targets)
    for lst in possible_targets:
        arrangements_iterators.append(Arrangements(range(len(lst[0])), lst[1]))

In [8]:
def find_conjugations(arrangements_iterators, possible_targets, source, Conj, f, g):
    for tup in product(*arrangements_iterators):
        current_target = []
        for i in range(len(tup)):
            for j in tup[i]:
                current_target.append(possible_targets[i][0][j])
        #print('current_target:', current_target)
        try:
            phi = f.domain().point_transformation_matrix(current_target, source)
            #print('phi:', phi)
            #print('phi inverse', phi.inverse())
            #print([phi*pnt for pnt in current_target])
            %time f.conjugate(phi) == g
            if f.conjugate(phi) == g:
                Conj.append(phi)
        except ValueError:
            pass

In [9]:
K.<v>=CyclotomicField(3)
P.<x,y,z> = ProjectiveSpace(K, 2)
f = DynamicalSystem([8*x^7 - 35*x^4*y^3 - 35*x^4*z^3 - 7*x*y^6 - 140*x*y^3*z^3 - 7*x*z^6, -7*x^6*y - 35*x^3*y^4 - 140*x^3*y*z^3 + 8*y^7 - 35*y^4*z^3 - 7*y*z^6,-7*x^6*z - 140*x^3*y^3*z - 35*x^3*z^4 - 7*y^6*z - 35*y^3*z^4 + 8*z^7])
g = f

In [10]:
L, K = calculate_periodic_points(f,g)

In [11]:
%%timeit
L, K = calculate_periodic_points(f,g)

436 ms ± 5.72 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
mult_to_point_L = {}
mult_to_point_K = {}
point_to_mult_L = {}
point_to_mult_K = {}
repeated_mult_L = {}
level = 0
initialize_dictionaries(f, g, L, K, mult_to_point_L, mult_to_point_K, point_to_mult_L, point_to_mult_K, repeated_mult_L)

In [13]:
%%timeit
mult_to_point_L = {}
mult_to_point_K = {}
point_to_mult_L = {}
point_to_mult_K = {}
repeated_mult_L = {}
level = 0
initialize_dictionaries(f, g, L, K, mult_to_point_L, mult_to_point_K, point_to_mult_L, point_to_mult_K, repeated_mult_L)

667 ms ± 19.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
source = []
corresponding = []
n = f.domain().dimension_relative()
more = find_independent_set(repeated_mult_L, source, corresponding)
more

False

In [15]:
%%timeit
source = []
corresponding = []
n = f.domain().dimension_relative()
more = find_independent_set(repeated_mult_L, source, corresponding)

7.73 ms ± 220 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
i_repeated_mult = deepcopy(repeated_mult_L)
a_repeated_mult = {}
found_no_more = True
while more:
    level += 1
    more, found_no_more, source, corresponding = (i_repeated_mult, point_to_mult_L, mult_to_point_K, f, g, mult_to_point_L, point_to_mult_K, repeated_mult_L, a_repeated_mult)
    if found_no_more:
        raise ValueError('no more rational preimages')
    i_repeated_mult = deepcopy(a_repeated_mult)
    a_repeated_mult = {}
    found_no_more = True

In [17]:
possible_targets = []
arrangements_iterators = []
initialize_interator(corresponding, possible_targets, mult_to_point_K, arrangements_iterators)

In [18]:
%%timeit
possible_targets = []
arrangements_iterators = []
initialize_interator(corresponding, possible_targets, mult_to_point_K, arrangements_iterators)

7.8 µs ± 52 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [54]:
Conj = []
find_conjugations(arrangements_iterators, possible_targets, source, Conj, f, g)

In [29]:
%%timeit
for tup in product(*arrangements_iterators):
    pass

4.96 s ± 19.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
arrangements_iterators[0].cardinality()

1413720

In [76]:
#%%timeit
tup = t[6]
current_target = []
for i in range(len(tup)):
    for j in tup[i]:
        current_target.append(possible_targets[i][0][j])
#print('current_target:', current_target)
try:
    %time phi = f.domain().point_transformation_matrix(current_target, source)
    #print('phi:', phi)
    #print('phi inverse', phi.inverse())
    #print([phi*pnt for pnt in current_target])
    #k = f.conjugate(phi)
    #bo = k == g
    %time k = f.conjugate(phi)
    %time bo = k == g
    if bo:
        Conj.append(phi)
except ValueError:
    pass

CPU times: user 4.91 ms, sys: 0 ns, total: 4.91 ms
Wall time: 4.91 ms
CPU times: user 16.3 ms, sys: 0 ns, total: 16.3 ms
Wall time: 15.9 ms
CPU times: user 500 µs, sys: 0 ns, total: 500 µs
Wall time: 502 µs


In [20]:
t = list(product(*arrangements_iterators))

In [21]:
sys.getsizeof(t)

11746976

In [37]:
len(t)

1413720

In [43]:
sys.getsizeof(t[2])

48

In [46]:
size = get_size(t)

In [44]:
import sys

def get_size(obj, seen=None):
    """Recursively finds size of objects"""
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    # Important mark as seen *before* entering recursion to gracefully handle
    # self-referential objects
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([get_size(v, seen) for v in obj.values()])
        size += sum([get_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        size += get_size(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_size(i, seen) for i in obj])
    return size

In [49]:
RR(size*10**-6)

351.039776000000

In [56]:
Conj

[]

In [22]:
def test_conj(tup):
    current_target = []
    for i in range(len(tup)):
        for j in tup[i]:
            current_target.append(possible_targets[i][0][j])
    try:
        phi = f.domain().point_transformation_matrix(current_target, source)
        if f.conjugate(phi) == g:
            return phi
    except ValueError:
        pass
    return None

In [23]:
test_conj(t[0])

[1 0 0]
[0 1 0]
[0 0 1]

In [24]:
from sage.parallel.use_fork import p_iter_fork

In [28]:
parallel_data = []
for tup in t:
    parallel_data.append(([tup], {}))

X = p_iter_fork(2)

conj = []
for ret in X(test_conj, parallel_data):
    print(ret[0])
    if ret[1]:
        conj.append(ret[1])

([([0, 1, 2, 3],)], {})
([([0, 1, 2, 4],)], {})
([([0, 1, 2, 5],)], {})
([([0, 1, 2, 6],)], {})
([([0, 1, 2, 7],)], {})
([([0, 1, 2, 8],)], {})
([([0, 1, 2, 9],)], {})
([([0, 1, 2, 10],)], {})
([([0, 1, 2, 11],)], {})
([([0, 1, 2, 12],)], {})
([([0, 1, 2, 13],)], {})
([([0, 1, 2, 14],)], {})
([([0, 1, 2, 15],)], {})
([([0, 1, 2, 16],)], {})
([([0, 1, 2, 17],)], {})
([([0, 1, 2, 18],)], {})
([([0, 1, 2, 19],)], {})
([([0, 1, 2, 20],)], {})
([([0, 1, 2, 21],)], {})
([([0, 1, 2, 22],)], {})
([([0, 1, 2, 23],)], {})
([([0, 1, 2, 24],)], {})
([([0, 1, 2, 25],)], {})
([([0, 1, 2, 26],)], {})
([([0, 1, 2, 27],)], {})
([([0, 1, 2, 28],)], {})
([([0, 1, 2, 29],)], {})
([([0, 1, 2, 30],)], {})
([([0, 1, 2, 31],)], {})
([([0, 1, 2, 32],)], {})
([([0, 1, 2, 33],)], {})
([([0, 1, 2, 34],)], {})
([([0, 1, 2, 35],)], {})
([([0, 1, 3, 2],)], {})
([([0, 1, 3, 4],)], {})
([([0, 1, 3, 5],)], {})
([([0, 1, 3, 6],)], {})
([([0, 1, 3, 7],)], {})
([([0, 1, 3, 8],)], {})
([([0, 1, 3, 9],)], {})
([([0, 1, 3, 1

([([0, 1, 11, 35],)], {})
([([0, 1, 12, 2],)], {})
([([0, 1, 12, 3],)], {})
([([0, 1, 12, 4],)], {})
([([0, 1, 12, 5],)], {})
([([0, 1, 12, 6],)], {})
([([0, 1, 12, 7],)], {})
([([0, 1, 12, 8],)], {})
([([0, 1, 12, 9],)], {})
([([0, 1, 12, 10],)], {})
([([0, 1, 12, 11],)], {})
([([0, 1, 12, 13],)], {})
([([0, 1, 12, 14],)], {})
([([0, 1, 12, 15],)], {})
([([0, 1, 12, 16],)], {})
([([0, 1, 12, 18],)], {})
([([0, 1, 12, 17],)], {})
([([0, 1, 12, 19],)], {})
([([0, 1, 12, 20],)], {})
([([0, 1, 12, 21],)], {})
([([0, 1, 12, 22],)], {})
([([0, 1, 12, 23],)], {})
([([0, 1, 12, 24],)], {})
([([0, 1, 12, 25],)], {})
([([0, 1, 12, 26],)], {})
([([0, 1, 12, 27],)], {})
([([0, 1, 12, 28],)], {})
([([0, 1, 12, 29],)], {})
([([0, 1, 12, 30],)], {})
([([0, 1, 12, 31],)], {})
([([0, 1, 12, 32],)], {})
([([0, 1, 12, 33],)], {})
([([0, 1, 12, 34],)], {})
([([0, 1, 12, 35],)], {})
([([0, 1, 13, 2],)], {})
([([0, 1, 13, 3],)], {})
([([0, 1, 13, 4],)], {})
([([0, 1, 13, 5],)], {})
([([0, 1, 13, 6],)], {})

Exception raised by child process with pid=23882:


KeyboardInterrupt: 

In [29]:
len(parallel_data)

1413720